# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('cal_housing_clean.csv')

In [3]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [4]:
df.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [6]:
x_data = df.drop(['medianHouseValue'],axis = 1)

In [7]:
y_data =df['medianHouseValue']

In [12]:
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size = 0.3,random_state = 101)

In [13]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(14448, 6)
(14448,)
(6192, 6)
(6192,)


### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [14]:
from sklearn.preprocessing import MinMaxScaler

In [15]:
scaler = MinMaxScaler()

In [16]:
scaler.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [17]:
x_train =pd.DataFrame(data=scaler.transform(x_train),columns=x_train.columns,index=x_train.index)

In [18]:
x_test =pd.DataFrame(data=scaler.transform(x_test),columns=x_train.columns,index=x_test.index)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [19]:
df.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [20]:
import tensorflow as tf

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [21]:
age = tf.feature_column.numeric_column('housingMedianAge')
rooms = tf.feature_column.numeric_column('totalRooms')
bedrooms = tf.feature_column.numeric_column('totalBedrooms')
pop = tf.feature_column.numeric_column('population')
households = tf.feature_column.numeric_column('households')
income = tf.feature_column.numeric_column('medianIncome')

In [22]:
feature_cols = [age,rooms,bedrooms,pop,households,income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [27]:
input_func = tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,
                                                                batch_size = 10,num_epochs = 1000,shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [28]:
model = tf.estimator.DNNRegressor(hidden_units=[6,6,6],
                                  feature_columns=feature_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_is_chief': True, '_protocol': None, '_master': '', '_save_checkpoints_steps': None, '_train_distribute': None, '_experimental_distribute': None, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f023fda64a8>, '_eval_distribute': None, '_task_type': 'worker', '_evaluation_master': '', '_tf_random_seed': None, '_service': None, '_device_fn': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_model_dir': '/tmp/tmprvzmuahh', '_keep_checkpoint_max': 5, '_task_id': 0, '_num_worker_replicas': 1}


In [29]:
model.train(input_fn=input_func,steps=25000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmprvzmuahh/model.ckpt.
INFO:tensorflow:loss = 202716610000.0, step = 1
INFO:tensorflow:global_step/sec: 390.414
INFO:tensorflow:loss = 637984050000.0, step = 101 (0.260 sec)
INFO:tensorflow:global_step/sec: 538.588
INFO:tensorflow:loss = 366370130000.0, step = 201 (0.183 sec)
INFO:tensorflow:global_step/sec: 557.439
INFO:tensorflow:loss = 413600380000.0, step = 301 (0.180 sec)
INFO:tensorflow:global_step/sec: 454.525
INFO:tensorflow:loss = 198703450000.0, st

INFO:tensorflow:loss = 153974590000.0, step = 6701 (0.212 sec)
INFO:tensorflow:global_step/sec: 465.153
INFO:tensorflow:loss = 103831090000.0, step = 6801 (0.216 sec)
INFO:tensorflow:global_step/sec: 458.371
INFO:tensorflow:loss = 106609840000.0, step = 6901 (0.219 sec)
INFO:tensorflow:global_step/sec: 464.367
INFO:tensorflow:loss = 112963120000.0, step = 7001 (0.215 sec)
INFO:tensorflow:global_step/sec: 453.166
INFO:tensorflow:loss = 90382500000.0, step = 7101 (0.221 sec)
INFO:tensorflow:global_step/sec: 454.315
INFO:tensorflow:loss = 144745960000.0, step = 7201 (0.221 sec)
INFO:tensorflow:global_step/sec: 466.533
INFO:tensorflow:loss = 82912730000.0, step = 7301 (0.215 sec)
INFO:tensorflow:global_step/sec: 450.97
INFO:tensorflow:loss = 106340246000.0, step = 7401 (0.220 sec)
INFO:tensorflow:global_step/sec: 439.13
INFO:tensorflow:loss = 34825290000.0, step = 7501 (0.228 sec)
INFO:tensorflow:global_step/sec: 464.563
INFO:tensorflow:loss = 86113330000.0, step = 7601 (0.214 sec)
INFO:te

INFO:tensorflow:loss = 185419580000.0, step = 14601 (0.221 sec)
INFO:tensorflow:global_step/sec: 466.782
INFO:tensorflow:loss = 130453170000.0, step = 14701 (0.215 sec)
INFO:tensorflow:global_step/sec: 460.466
INFO:tensorflow:loss = 65635460000.0, step = 14801 (0.215 sec)
INFO:tensorflow:global_step/sec: 464.413
INFO:tensorflow:loss = 53810414000.0, step = 14901 (0.215 sec)
INFO:tensorflow:global_step/sec: 463.723
INFO:tensorflow:loss = 208018540000.0, step = 15001 (0.218 sec)
INFO:tensorflow:global_step/sec: 463.783
INFO:tensorflow:loss = 103854810000.0, step = 15101 (0.213 sec)
INFO:tensorflow:global_step/sec: 465.581
INFO:tensorflow:loss = 117458910000.0, step = 15201 (0.221 sec)
INFO:tensorflow:global_step/sec: 447.351
INFO:tensorflow:loss = 55215660000.0, step = 15301 (0.218 sec)
INFO:tensorflow:global_step/sec: 474.208
INFO:tensorflow:loss = 39723700000.0, step = 15401 (0.211 sec)
INFO:tensorflow:global_step/sec: 457.486
INFO:tensorflow:loss = 134398580000.0, step = 15501 (0.220 

INFO:tensorflow:global_step/sec: 426.887
INFO:tensorflow:loss = 65388260000.0, step = 22501 (0.233 sec)
INFO:tensorflow:global_step/sec: 388.665
INFO:tensorflow:loss = 117462630000.0, step = 22601 (0.258 sec)
INFO:tensorflow:global_step/sec: 420.121
INFO:tensorflow:loss = 38983786000.0, step = 22701 (0.238 sec)
INFO:tensorflow:global_step/sec: 456.63
INFO:tensorflow:loss = 226048280000.0, step = 22801 (0.218 sec)
INFO:tensorflow:global_step/sec: 445.848
INFO:tensorflow:loss = 114428666000.0, step = 22901 (0.226 sec)
INFO:tensorflow:global_step/sec: 465.945
INFO:tensorflow:loss = 66342010000.0, step = 23001 (0.214 sec)
INFO:tensorflow:global_step/sec: 412.2
INFO:tensorflow:loss = 191447140000.0, step = 23101 (0.244 sec)
INFO:tensorflow:global_step/sec: 452.335
INFO:tensorflow:loss = 199905180000.0, step = 23201 (0.221 sec)
INFO:tensorflow:global_step/sec: 434.701
INFO:tensorflow:loss = 84157506000.0, step = 23301 (0.229 sec)
INFO:tensorflow:global_step/sec: 436.506
INFO:tensorflow:loss 

##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [31]:
predict_input_func = tf.estimator.inputs.pandas_input_fn(x = x_test,y=y_test
                                                        ,shuffle=False
                                                        ,batch_size=10,
                                                        num_epochs=1)

In [32]:
preds = model.predict(input_fn=predict_input_func)

In [33]:
preds = list(preds)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmprvzmuahh/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [34]:
final_preds = []
for pred in preds:
    final_preds.append(pred['predictions'])

In [35]:
from sklearn.metrics import mean_squared_error

In [37]:
## making it Root Mean Squared by taking Squared root of the returned value

In [38]:
mean_squared_error(y_test,final_preds)**0.5

97003.42313848576

# Great Job!